In [1]:
# All library calls go here
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc #mamba install -c conda-forge netCDF4 # conda install -c conda-forge netcdf4

import planetary_computer
import xarray as xr
import fsspec
import pystac_client
import datetime as dtt

import requests

from matplotlib import pyplot

import dask_gateway

import pickle

import os, uuid
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient,ContentSettings
import io